In [0]:
!unzip com.zip

Archive:  com.zip
   creating: com/
  inflating: __MACOSX/._com          
  inflating: com/07file_52.jpg       
  inflating: __MACOSX/com/._07file_52.jpg  
  inflating: com/07file_46.jpg       
  inflating: __MACOSX/com/._07file_46.jpg  
  inflating: com/10file_3.jpg        
  inflating: __MACOSX/com/._10file_3.jpg  
  inflating: com/08file_27.jpg       
  inflating: __MACOSX/com/._08file_27.jpg  
  inflating: com/07file_91.jpg       
  inflating: __MACOSX/com/._07file_91.jpg  
  inflating: com/08file_33.jpg       
  inflating: __MACOSX/com/._08file_33.jpg  
  inflating: com/07file_85.jpg       
  inflating: __MACOSX/com/._07file_85.jpg  
  inflating: com/03file_13.jpg       
  inflating: __MACOSX/com/._03file_13.jpg  
  inflating: com/02file_4.jpg        
  inflating: __MACOSX/com/._02file_4.jpg  
  inflating: com/04file_48.jpg       
  inflating: __MACOSX/com/._04file_48.jpg  
  inflating: com/04file_74.jpg       
  inflating: __MACOSX/com/._04file_74.jpg  
  inflating: com/04file_60

In [0]:
from scipy import ndimage, misc
import imageio
from PIL import Image
import os
import glob
import numpy as np

def return_dataset():

  folder = './com/'
  all_data_dirlist = list(os.listdir(folder))
  # print(all_data_dirlist)
  X_train = np.empty(0, dtype=np.float32)

  count = 0

  for index, img_filename in enumerate(all_data_dirlist):
    image = Image.open(os.path.join(folder, img_filename)).convert('L', (0.2989, 0.5870, 0.1140, 0))
    if (np.random.rand() < 0.5):
      image = image.transpose(Image.FLIP_LEFT_RIGHT)
    image_resized = image.resize((112, 112))
    image = np.asarray(image_resized)

    mean = 0.0   # some constant
    std = 1.0    # some constant (standard deviation)
    image = image + np.random.normal(mean, std, image.shape)
    image = np.clip(image, 0, 255)

    image = image.reshape((112, 112))
    # print(image.shape)
    image = image.flatten()
    # print(len(image))
    # print(image)
    X_train = np.append(X_train, image)

  X_train = X_train.reshape((1000, 112, 112))
  return X_train

In [0]:
X_train = return_dataset()
X_train.shape

(1000, 112, 112)

In [0]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 112
        self.img_cols = 112
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 28 * 28, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((28, 28, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        # (X_train, _), (_, _) = mnist.load_data()

        global X_train

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                # Sample generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                # Train the critic
                d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                                [valid, fake, dummy])

            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.generator_model.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("./arch_%d.png" % epoch)
        plt.close()
    

In [0]:
wgan = WGANGP()
wgan.train(epochs=30000, batch_size=32, sample_interval=100)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100352)            10135552  
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28, 128)       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 128)       262272    
_________________________________________________________________
batch_normalization_6 (Batch (None, 56, 56, 128)       512       
_________________________________________________________________
activation_4 (Activation)    (None, 56, 56, 128)       0         
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 112, 112, 128)    

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 17.649664] [G loss: 1.237347]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 14.669161] [G loss: 0.734855]
2 [D loss: 13.209162] [G loss: 0.426467]
3 [D loss: 9.560051] [G loss: 0.473677]
4 [D loss: 8.438645] [G loss: -0.271268]
5 [D loss: 7.683563] [G loss: -0.399930]
6 [D loss: 5.022382] [G loss: 0.007056]
7 [D loss: 3.502680] [G loss: -0.563130]
8 [D loss: 3.408469] [G loss: -0.567703]
9 [D loss: 2.646380] [G loss: -0.599800]
10 [D loss: 1.905118] [G loss: -1.059621]
11 [D loss: 1.472359] [G loss: -1.283830]
12 [D loss: 1.233490] [G loss: -1.157141]
13 [D loss: 0.847447] [G loss: -1.245679]
14 [D loss: 0.788646] [G loss: -1.687223]
15 [D loss: 0.011506] [G loss: -1.807694]
16 [D loss: 0.026077] [G loss: -2.071390]
17 [D loss: -0.559265] [G loss: -2.020180]
18 [D loss: -0.243721] [G loss: -2.447014]
19 [D loss: -0.509847] [G loss: -2.419567]
20 [D loss: -0.761243] [G loss: -2.537344]
21 [D loss: -0.424177] [G loss: -2.593389]
22 [D loss: -0.565658] [G loss: -2.537954]
23 [D loss: -0.994324] [G loss: -2.711644]
24 [D loss: -1.282327] [G loss: -2.557

In [0]:
r, c = 5, 5
noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)

# Rescale images 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
        axs[i,j].axis('off')
        cnt += 1

NameError: ignored